In [1]:
! pip install llama-index == 0.11.13 
! pip install llama-index-llms-ollama
! pip install llama-index-embeddings-ollama

zsh:1: = not found


In [2]:
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)

In [3]:
import os
ollama_host = os.getenv("OLLAMA_API", "http://localhost:11434")
ollama_model = 'llama3.2-vision:latest'
logger.info(f"ollama_host: {ollama_host}")
logger.info(f"ollama_model: {ollama_model}")

INFO:__main__:ollama_host: http://localhost:11434
INFO:__main__:ollama_model: llama3.2:latest


In [5]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

llm = Ollama(model=ollama_model, base_url=ollama_host)

resp = llm.complete(prompt="Hello, world!")
logger.info(resp)

embedding = OllamaEmbedding(model_name=ollama_model, base_url=ollama_host)
Settings.embed_model = embedding
Settings.llm = llm
print(llm.complete(prompt="Hello, world!"))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:__main__:Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?


> Document Nodes

In [6]:
from llama_index.core import Document
from llama_index.core.schema import TextNode

doc = Document(text="This is a sample document text")
n1 = TextNode(text=doc.text[0: 16])
n2 = TextNode(text=doc.text[17: 30])
logger.info(n1)
logger.info(n2)

INFO:__main__:Node ID: 23dfb79f-ce68-4556-ab4f-3867e576915e
Text: This is a sample
INFO:__main__:Node ID: 7afffc8a-3938-49d7-a21b-25b4a11c3be3
Text: document text


> Document Splitter 

In [ ]:
from llama_index.core import Document
from llama_index.core.schema import TextNode 
from llama_index.core.node_parser import TokenTextSplitter
doc = Document(
    text=("this is sentence 1. this is sentence 2. " "this is sentence 3."),
    metadata={"author": "John Smith"}
    )

splitter = TokenTextSplitter(chunk_size=12, chunk_overlap=0, separator=" ")
nodes = splitter.get_nodes_from_documents([doc])
for node in nodes:
    print(f'node: {node}')
    print(f'metadata: {node.metadata}')


> Node Relationship
```
* Enable more contextual querying
* Allows tracking provenance
* Enables navigation through the nodes
* Supports the construction of knowledge graphs
* Improves the index structure
```

In [ ]:
from llama_index.core import Document
from llama_index.core.schema import TextNode, NodeRelationship
from llama_index.core.node_parser import TokenTextSplitter
doc = Document(
    text=("first sentence. second sentence."),
    metadata={"author": "John Smith"}
    )
n1 = TextNode(text="first sentence.", node_id=doc.doc_id)
n2 = TextNode(text="second sentence.", node_id=doc.doc_id)

n1.relationships[NodeRelationship.NEXT] = n2.node_id
n2.relationships[NodeRelationship.PREVIOUS] = n1.node_id

print(n1.relationships)
print(n2.relationships)


> Summary Index


In [ ]:
from llama_index.core.settings import Settings

from llama_index.core.schema import TextNode
from llama_index.core import SummaryIndex

nodes = [
  TextNode(text="Lionel Messi's hometown is Rosario."),
  TextNode(text="He was born on June 24, 1987.")
]
index = SummaryIndex(nodes)
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query(
    "What is Messi's hometown?"
)
print(response)

In [ ]:
! pip install llama-index-readers-wikipedia
! pip install wikipedia

> Reading wiki content


In [8]:
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.core import Document, SummaryIndex
from llama_index.core.node_parser import SimpleNodeParser

reader = WikipediaReader()
docs = reader.load_data(pages=["https://en.wikipedia.org/wiki/Lionel_Messi"])
print(docs)
parser = SimpleNodeParser.from_defaults()
nodes = parser.get_nodes_from_documents(docs)
index = SummaryIndex(nodes)
query_engine = index.as_query_engine(llm=llm)
print("Ask me anything about Lionel Messi!")
response = query_engine.query("What is Messi's hometown?")
print(response)


[Document(id_='592784', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Rosario (Spanish pronunciation: [roˈsaɾjo]) is the largest city in the central Argentine province of Santa Fe. The city, located 300 km (186 mi) northwest of Buenos Aires on the west bank of the Paraná River, is the third-most populous city in the country after Buenos Aires and Cordoba. With a growing and important metropolitan area, Greater Rosario has an estimated population of 1,750,000 as of 2020. One of its main attractions includes the neoclassical, Art Nouveau, and Art Deco architecture that has been preserved in hundreds of residences, houses and public buildings. The city is also famous for being the birthplace of the legendary Argentine footballer Lionel Messi.\nRosario is the head city of the Rosario Department and is located at the heart of the major industrial corridor in Argentina. The city is a major railroad terminal and the shippi